In [1]:
%pwd

'd:\\recommandation\\MLproject\\research'

In [2]:
%cd ..

d:\recommandation\MLproject


d:\recommandation\rithvik\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [9]:
lib = %pwd
lib

'd:\\recommandation\\MLproject'

In [10]:
import sys

sys.path.append(lib)

In [14]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [12]:
import os
import urllib.request as request
import zipfile
from src.mlProject import logger

In [15]:
from pathlib import Path

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE):

        config_filepath = Path(config_filepath)

        self.config = read_yaml(config_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-11 16:26:18,020: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-11 16:26:18,023: INFO: common: created directory at: artifacts]
[2024-05-11 16:26:18,025: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-11 16:26:40,081: INFO: 3052114499: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 24857
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "e72f5befdc6bbe5f0ff08c13d10e7047157387c134ad608b9200dee366dd97d8"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7180:1114EB:219843:2BC52E:663F4EE7
Accept-Ranges: bytes
Date: Sat, 11 May 2024 10:56:40 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10230-MAA
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1715425000.189770,VS0,VE296
Vary: Authorization,Accept-E

In [1]:
%pwd

'd:\\recommandation\\MLproject\\research'

In [2]:
%cd ..

d:\recommandation\MLproject


d:\recommandation\rithvik\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [9]:
import pandas as pd

df = pd.read_csv("artifacts/data_ingestion/olist_customers_dataset.csv")

In [10]:
features_to_drop = [
    'order_purchase_timestamp','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date',
    'customer_zip_code_prefix','shipping_limit_date','product_weight_g','product_length_cm','product_height_cm','product_width_cm',
    'order_id', 'customer_id', 'payment_sequential','customer_unique_id', 'order_item_id', 'product_id',
    'seller_id','review_comment_message','product_category_name','product_name_lenght']


In [11]:
df = df.drop(columns=features_to_drop)

In [12]:
df.columns

Index(['order_status', 'payment_type', 'payment_installments', 'payment_value',
       'customer_city', 'customer_state', 'price', 'freight_value',
       'product_description_lenght', 'product_photos_qty',
       'product_category_name_english', 'review_score'],
      dtype='object')

In [13]:
features = ['order_status', 'payment_type', 'payment_installments', 'payment_value', 'customer_city', 'customer_state', 'price', 'freight_value', 'product_description_lenght', 'product_photos_qty', 'product_category_name_english', 'Time_taken_for_delivery', 'delivery_delay']

In [26]:
input_data = ['delivered','credit_card',3,18.59,'florianopolis','MG',339.0,7.78,664.0,6,'cool_stuff',5,-2]

In [21]:
from sklearn.preprocessing import LabelEncoder

categorical_features = [ 'order_status', 'product_category_name_english', 'customer_city', 'customer_state', 'payment_type' ]

In [22]:
label_encoder = LabelEncoder()
for col in categorical_features:
    df[col] = label_encoder.fit_transform(df[col])

In [24]:
df['product_category_name_english'].unique()

array([49, 20, 39, 50,  7, 37, 15, 42, 66, 70, 43, 51,  5, 59, 24, 65, 54,
        6, 16, 17, 48, 63,  8, 26, 25, 47, 69, 60, 55, 68, 12, 53, 46, 28,
       56, 36, 21,  2, 40, 18, 38, 57, 14, 10, 35,  3,  1, 67, 19, 45, 58,
       34, 13, 31, 52, 62,  4, 33, 32, 29, 44,  0, 11, 30, 27, 64, 22, 41,
       23,  9, 61])

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Separate features and target variable
X = df.drop(columns=['review_score'])
y = df['review_score']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define and train XGBoost regression model
model = XGBRegressor()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 1.5971275212582638


In [27]:
model.feature_names_in_

array(['order_status', 'payment_type', 'payment_installments',
       'payment_value', 'customer_city', 'customer_state', 'price',
       'freight_value', 'product_description_lenght',
       'product_photos_qty', 'product_category_name_english'],
      dtype='<U29')

In [ ]:
input_data = ['delivered','credit_card',3,18.59,'florianopolis','MG',339.0,7.78,664.0,6,'cool_stuff',5,-2]